In [179]:
import pandas as pd
import numpy as np
import random
import time
import warnings
warnings.filterwarnings("ignore")
courses= pd.ExcelFile("courses.xlsx")


In [180]:
def MAD(x):
    '''
    calculates the median absolute deviation which is defined as median(|x_i - median(x)|
    inputs: arraylike 
          X
    outputs: numpy float
          MAD
    '''
    return np.median(np.absolute(x - np.median(x)))

def isTaken(course):
    '''
    checks whether the courses is already in the time table or not
    inputs: str
          course name
    outputs: bool
           True if exists in the table, False otherwise
    '''
    t1=any(timeTable["course1"].astype(str).str.match(course))
    t2=any(timeTable["course2"].astype(str).str.match(course))
    t3=any(timeTable["course3"].astype(str).str.match(course))

    return t1 or t2 or t3


def isIntersected(*args):
    '''
    checks the intersection of students between two courses
    inputs: str
          course 1 name , course 2 name
    outputs: bool
           True if there is intersection and False otherwise
    '''

    df_merge=pd.concat(list(args))
    isduplicated=df_merge.duplicated()
    return not len(df_merge[isduplicated])==0


def isInCourse(course,M):
    '''
    checks whether stuent M is in course Course or not

    '''

    try:
        return coursesDic2[course].loc[students.index[M]].count()>0
    except:
        return False

def getStudentCourses(M):
    '''
    returns student M courses

    '''
    studentMCourses=[]
    for course in coursesDic2.keys():
        if isInCourse(course,M):
            studentMCourses.append(course)
        return studentMCourses



def studentMMAD(M):
    '''
    calculates the MAD of student M's time table

    '''
    studentMTimeTable=timeTable[timeTable["course1"].astype(str).str.match("|".join(getStudentCourses(M))) | 
                      timeTable["course2"].astype(str).str.match("|".join(getStudentCourses(M))) |
                      timeTable["course3"].astype(str).str.match("|".join(getStudentCourses(M)))]
    index=studentMTimeTable.index
    return MAD(index)   

def avgMAD():
  '''
  calculates avgerage Mad for this particular time table
  '''
  
    sum=0
    for M in range(len(students)):
        sum+=studentMMAD(M)

    avg=sum/len(students)
    return avg

In [182]:
start_time = time.time()

timeTableList=[]
for iteration in range(30):
    coursesDic=dict()
    for sheetnum,sheet in enumerate(courses.sheet_names):

        # Parse data from each worksheet as a Pandas DataFrame and append it 
        # to the dic
        df = courses.parse(sheet)
        df.set_index('name',inplace=True)
        df["name_col"]=df.index
        coursesDic[courses.sheet_names[sheetnum]]=df

        coursesDic2=coursesDic.copy()


    timeTable = pd.DataFrame(data={'course1':[], 'course2': [],'course3':[]})

    for i in range(1000):
        try:
            randCourseI=random.choice(list(coursesDic.keys()))
            randCourseJ=random.choice(list(coursesDic.keys()))
            randCourseK=random.choice(list(coursesDic.keys()))
        except:
            None

        if(not isTaken(randCourseI) and  not isTaken(randCourseJ)and  not isTaken(randCourseK)):
            if(isIntersected(coursesDic2[randCourseI],coursesDic2[randCourseJ],coursesDic2[randCourseK])==False):


                dayNCourses = pd.DataFrame([[randCourseI,randCourseJ,randCourseK]],
                                          columns=timeTable.columns)
                timeTable=timeTable.append(dayNCourses,ignore_index=True)
                coursesDic.pop(randCourseI)
                coursesDic.pop(randCourseJ)
                coursesDic.pop(randCourseK)



    for i in range(500):
        try:
            randCourseI=random.choice(list(coursesDic.keys()))
            randCourseJ=random.choice(list(coursesDic.keys()))
        except:
            None

        if(not isTaken(randCourseI) and  not isTaken(randCourseJ)):
            if(isIntersected(coursesDic2[randCourseI],coursesDic2[randCourseJ])==False):


                dayNCourses = pd.DataFrame([[randCourseI,randCourseJ,"-"]],
                                          columns=timeTable.columns)
                timeTable=timeTable.append(dayNCourses,ignore_index=True)
                coursesDic.pop(randCourseI)
                coursesDic.pop(randCourseJ)


    for course in list(coursesDic.keys()):

            coursesDic.pop(course)

            dayNCourses = pd.DataFrame([[course,"-","-"]],
                                      columns=timeTable.columns)
            timeTable=timeTable.append(dayNCourses,ignore_index=True)

    timeTableList.append(timeTable)

#Finding the table with minimum number of days 
for i in range(len(timeTableList)):
    if len(timeTableList[i])<len(timeTable):
        timeTable=timeTableList[i]

#Creating a students dataframe from all courses
students=pd.DataFrame(columns=['name'])
students.set_index('name',inplace=True)
students["e"]=0

for course in coursesDic2.values():

    students=pd.merge(students,course, how='outer',left_index=True, right_index=True).dropna(axis=1)


#shuffling and choosing the timetable with the maximum 

#average median absolute deviation
avgMADList=[]
timeTableList=[]
for i in range(100):
    timeTable=timeTable.sample(frac = 1,ignore_index=True)
    timeTableList.append(timeTable)
    avgMADList.append(avgMAD())
    

maxIndex=avgMADList.index(max(avgMADList))
timeTable=timeTableList[maxIndex]

In [ ]:
timeTable

In [184]:
end_time=time.time()
print(f"run time: {(end_time - start_time)/60} Minutes")


run time: 2.170034190018972 Minutes


In [6]:
import pandas as pd
import numpy as np
import random
import time
import warnings
warnings.filterwarnings("ignore")
class Scheduler():


    
    def __init__(self,courses):
        self.courses= courses 

    def MAD(self,x):
        '''
        calculates the median absolute deviation which is defined as median(|x_i - median(x)|
        inputs: arraylike 
              X
        outputs: numpy float
              MAD
        '''
        return np.median(np.absolute(x - np.median(x)))

    def isTaken(self,course):
        '''
        checks whether the courses is already in the time table or not
        inputs: str
              course name
        outputs: bool
               True if exists in the table, False otherwise
        '''
        t1=any(timeTable["course1"].astype(str).str.match(course))
        t2=any(timeTable["course2"].astype(str).str.match(course))
        t3=any(timeTable["course3"].astype(str).str.match(course))

        return t1 or t2 or t3


    def isIntersected(self,*args):
        '''
        checks the intersection of students between two courses
        inputs: str
              course 1 name , course 2 name
        outputs: bool
               True if there is intersection and False otherwise
        '''

        df_merge=pd.concat(list(args))
        isduplicated=df_merge.duplicated()
        return not len(df_merge[isduplicated])==0


    def isInCourse(self,course,M):
        '''
        checks whether stuent M is in course Course or not

        '''

        try:
            return coursesDic2[course].loc[students.index[M]].count()>0
        except:
            return False

    def getStudentCourses(self,M):
        '''
        returns student M courses

        '''
        studentMCourses=[]
        for course in coursesDic2.keys():
            if self.isInCourse(course,M):
                studentMCourses.append(course)
            return studentMCourses



    def studentMMAD(self,M):
        '''
        calculates the MAD of student M's time table

        '''
        studentMTimeTable=timeTable[timeTable["course1"].astype(str).str.match("|".join(self.getStudentCourses(M))) | 
                          timeTable["course2"].astype(str).str.match("|".join(self.getStudentCourses(M))) |
                          timeTable["course3"].astype(str).str.match("|".join(self.getStudentCourses(M)))]
        index=studentMTimeTable.index
        return self.MAD(index)   

    def avgMAD(self):
        '''
        calculates avgerage Mad for this particular time table
        '''

        sum=0
        for M in range(len(students)):
            sum+=self.studentMMAD(M)

        avg=sum/len(students)
        return avg

    
    def getTable(self):
        timeTableList=[]
        for iteration in range(30):
            coursesDic=dict()
            for sheetnum,sheet in enumerate(self.courses.sheet_names):

                # Parse data from each worksheet as a Pandas DataFrame and append it 
                # to the dic
                df = self.courses.parse(sheet)
                df.set_index('name',inplace=True)
                df["name_col"]=df.index
                coursesDic[self.courses.sheet_names[sheetnum]]=df
                
                global coursesDic2
                coursesDic2=coursesDic.copy()

            global timeTable
            timeTable = pd.DataFrame(data={'course1':[], 'course2': [],'course3':[]})

            for i in range(10):
                try:
                    randCourseI=random.choice(list(coursesDic.keys()))
                    randCourseJ=random.choice(list(coursesDic.keys()))
                    randCourseK=random.choice(list(coursesDic.keys()))
                except:
                    None

                if(not self.isTaken(randCourseI) and  not self.isTaken(randCourseJ)and  not self.isTaken(randCourseK)):
                    if(self.isIntersected(coursesDic2[randCourseI],coursesDic2[randCourseJ],coursesDic2[randCourseK])==False):


                        dayNCourses = pd.DataFrame([[randCourseI,randCourseJ,randCourseK]],
                                                  columns=timeTable.columns)
                        timeTable=timeTable.append(dayNCourses,ignore_index=True)
                        coursesDic.pop(randCourseI)
                        coursesDic.pop(randCourseJ)
                        coursesDic.pop(randCourseK)



            for i in range(5):
                try:
                    randCourseI=random.choice(list(coursesDic.keys()))
                    randCourseJ=random.choice(list(coursesDic.keys()))
                except:
                    None

                if(not self.isTaken(randCourseI) and  not self.isTaken(randCourseJ)):
                    if(self.isIntersected(coursesDic2[randCourseI],coursesDic2[randCourseJ])==False):


                        dayNCourses = pd.DataFrame([[randCourseI,randCourseJ,"-"]],
                                                  columns=timeTable.columns)
                        timeTable=timeTable.append(dayNCourses,ignore_index=True)
                        coursesDic.pop(randCourseI)
                        coursesDic.pop(randCourseJ)


            for course in list(coursesDic.keys()):

                    coursesDic.pop(course)

                    dayNCourses = pd.DataFrame([[course,"-","-"]],
                                              columns=timeTable.columns)
                    timeTable=timeTable.append(dayNCourses,ignore_index=True)

            timeTableList.append(timeTable)

        #Finding the table with minimum number of days 
        for i in range(len(timeTableList)):
            if len(timeTableList[i])<len(timeTable):
                timeTable=timeTableList[i]

        #Creating a students dataframe from all courses
        global students
        students=pd.DataFrame(columns=['name'])
        students.set_index('name',inplace=True)
        students["e"]=0

        for course in coursesDic2.values():

            students=pd.merge(students,course, how='outer',left_index=True, right_index=True).dropna(axis=1)


        #shuffling and choosing the timetable with the maximum 

        #average median absolute deviation
        avgMADList=[]
        timeTableList=[]
        for i in range(100):
            timeTable=timeTable.sample(frac = 1,ignore_index=True)
            timeTableList.append(timeTable)
            avgMADList.append(self.avgMAD())


        maxIndex=avgMADList.index(max(avgMADList))
        timeTable=timeTableList[maxIndex]

        return timeTable

In [7]:
Scheduler(pd.ExcelFile("courses.xlsx")).getTable()

,course1,course2,course3
0,ManufSystemAutomation,Thermodynamics,Mechanics1
1,Microcontroller,-,-
2,Ethics,-,-
3,DigitalControl,-,-
4,Control1,Robotics,-
5,MechanicsAndThermodynamics,-,-
6,MachineDesign2,FundOfMangement,-
7,Electronics1,-,-
8,PLC,-,-
9,Calc3,-,-
